In [1]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import numpy as np
import os
from random import shuffle

In [2]:
from tqdm import tqdm

In [3]:
TRAIN_DIR = r'Train'
TEST_DIR  = r'Test'

In [4]:
IMG_SIZE = 60
LR = 1e-3
MODEL_NAME = 'hand-{}-{}.model'.format(LR, '6conv-basic')

In [5]:
def label_img(img):
    word_label = img[10]
    if word_label == '8':return [1,0]
    elif word_label == '3':return[0,1]

In [6]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE) , (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    #np.save('train_data.npy',training_data)
    return training_data
                         

In [7]:
def process_test_data():
    testing_data = []
    i = 0
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = i
        i = i + 1
        if i == 200:
            i = 0
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img),img_num])
    np.save('test_data.npy',testing_data)
    return testing_data
    

In [8]:
traing_data = create_train_data()

100%|██████████| 400/400 [00:00<00:00, 732.98it/s]


In [9]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')
train = traing_data[:-50]
test = traing_data[-50:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]
Y = np.array(Y)
test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]
test_y = np.array(test_y)

In [10]:
import tensorflow as tf

from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1) ))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2,activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])

history = model.fit(X, Y, epochs=3, validation_data=(test_x, test_y))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 58, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 23328)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                746528    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 756,162
Trainable params: 756,162
Non-trainable params: 0
__________________________________________________